# Python Refresher Lab — FAANG-Level Hands-On

**Goal:** Refresh Python fundamentals + interview patterns the way FAANG expects ML engineers to code.

**Outcome:** You can write clean, efficient Python, explain complexity, and avoid classic interview gotchas.

---

## How to Start
1. Open this notebook in Colab
2. Complete all **TODO** sections
3. Restart runtime → Run all

---

## Lab Rules (FAANG Style)
- ✅ Complexity for each core function
- ✅ Handle edge cases
- ✅ Prefer O(n) over O(n²) when feasible
- ✅ Add tests
- ❌ No hardcoding outputs


In [2]:
from __future__ import annotations
from collections import Counter, defaultdict, deque
import heapq

# NOTE: Use this helper for quick sanity checks
def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

## Section 1 — Python Idioms & Core Data Structures

### Task 1.1: Comprehensions (no unnecessary loops)

Create (without loops):
- `even_squares`: squares of even numbers from 1..20
- `pairs`: list of (x, x*x) for x=1..10
- `flattened`: flatten `nested`

**Explain:** Why can comprehensions be faster than manual loops?

**Answer:** Because comprehensions run most of the work in optimized C code inside CPython and avoid a lot of Python-level loop overhead (extra bytecode, repeated lookups, method calls).

In [7]:
# TODO:
# Create even_squares
# HINT: range(1, 21), filter even with x % 2 == 0
even_squares = [x * x for x in range(1, 21) if x % 2 == 0]
# print(even_squares)

# TODO:
# Create pairs
pairs = [(x, x * x) for x in range(1, 11)]
print(pairs)

# TODO:
# Flatten nested
nested = [[1, 2], [3, 4], [5, 6]]
# HINT: nested loops inside a single comprehension
flattened = [ele for x in nested for ele in x]

check('even_squares', even_squares == [4, 16, 36, 64, 100, 144, 196, 256, 324, 400])
check('pairs', pairs[:3] == [(1, 1), (2, 4), (3, 9)] and pairs[-1] == (10, 100))
check('flattened', flattened == [1, 2, 3, 4, 5, 6])

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25), (6, 36), (7, 49), (8, 64), (9, 81), (10, 100)]
OK: even_squares
OK: pairs
OK: flattened


### Task 1.2: Counter (frequency counting)

Given `s`, compute:
- character frequency using `Counter`
- top 2 most common characters

**Interview angle:** When is Counter preferable to sorting?

**Answer:** Counter is preferable when you only need frequencies or the top-k, it is faster and cleaner: counting is O(n) and most_common(k) avoids sorting the whole list.

In [11]:
s = 'mississippi'

# TODO:
freq = Counter(s)
top2 = freq.most_common(2)

print(freq)
print(top2)
check('top2', top2[0][0] == 'i' and top2[0][1] == 4)

Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
[('i', 4), ('s', 4)]
OK: top2


### Task 1.3: defaultdict (grouping)

Group words by first letter.

**HINT:** `defaultdict(list)`

**Gotcha:** Don’t use a normal dict unless you handle missing keys.

In [15]:
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry', 'avocado']

# TODO:
groups = defaultdict(list)
for i in words:
  if i[0] == 'a':
    # print(i[0])
    groups[i[0]].append(i)

print(dict(groups))
check('groups[a]', dict(groups)['a'] == ['apple', 'apricot', 'avocado'])

{'a': ['apple', 'apricot', 'avocado']}
OK: groups[a]


## Section 2 — FAANG Core Patterns

### Task 2.1: Two Sum (Hash Map Pattern)

**FAANG Classic.** Return indices of two numbers that sum to target.

# HINT:
- Use a dict: value -> index
- For each `x`, check if `target-x` has been seen

**Explain:** Why is this O(n) time?

**Answer:** Because the array is scanned only once, and each step does O(1) average-time hash map operations (lookup + insert). So total work is proportional to n hence O(n).

In [17]:
def two_sum(nums, target):
    # TODO
    hashset = {}

    for i in range(len(nums)):
      diff = target - nums[i]
      if diff in hashset:
        current_idx = i
        existing_idx = hashset[diff]
        return[existing_idx, current_idx]
      else:
        hashset[nums[i]] = i

check('two_sum_1', two_sum([2, 7, 11, 15], 9) == [0, 1])
check('two_sum_2', two_sum([3, 2, 4], 6) == [1, 2])
check('two_sum_dup', two_sum([3, 3], 6) == [0, 1])

# Time Complexity: O(n)

OK: two_sum_1
OK: two_sum_2
OK: two_sum_dup


### Task 2.2: Longest Substring Without Repeating Characters (Sliding Window)

**FAANG Classic/Gotcha:** naive O(n²) solutions time out.

# HINT:
- Keep a map char -> most recent index
- Maintain a window `[l..r]` with unique chars

**Explain:** What invariant does the window maintain?

**Answer:** Invariant: the current window s[left:right+1] always contains no duplicate characters i.e., s_set is exactly the set of chars in the window, and all unique.

In [21]:
def length_of_longest_substring(s: str) -> int:
    # TODO
    l_s = list(s)
    s_set = set()

    left = 0
    max_length = 0

    for right in range(len(l_s)):
      while l_s[right] in s_set:
        s_set.remove(l_s[left])
        left += 1
      s_set.add(l_s[right])
      # print(s_set)

      max_length = max(max_length, right-left+1)
    return max_length

check('ls_abcabcbb', length_of_longest_substring('abcabcbb') == 3)
check('ls_bbbbb', length_of_longest_substring('bbbbb') == 1)
check('ls_pwwkew', length_of_longest_substring('pwwkew') == 3)
check('ls_empty', length_of_longest_substring('') == 0)

# Time Complexity: O(n)

OK: ls_abcabcbb
OK: ls_bbbbb
OK: ls_pwwkew
OK: ls_empty


### Task 2.3: Product of Array Except Self (Prefix/Suffix)

**FAANG Classic/Gotcha:**
- No division
- O(n) time

# HINT:
- Build prefix products in one pass
- Multiply by suffix products in reverse pass

**Explain:** Why does this work with zeros?

**Answer:** We never divide. We just multiply everything on the left and everything on the right.
If there is exactly 1 zero:
  - At the zero’s index: the product excludes the zero, so it’s the product of all non-zero numbers = non-zero value
  - At every other index: the product includes that single zero = 0
If there are 2+ zeros, every position’s left×right includes at least one zero = all 0s.

In [23]:
def product_except_self(nums):
    # TODO
    n = len(nums)
    res = [1] * n

    prefix = 1
    for i in range(n):
      res[i] = prefix
      prefix *= nums[i]

    suffix = 1
    for i in range(n-1, -1, -1):
      res[i] *= suffix
      suffix *= nums[i]
    return res


check('pes_1234', product_except_self([1, 2, 3, 4]) == [24, 12, 8, 6])
check('pes_zero', product_except_self([-1, 1, 0, -3, 3]) == [0, 0, 9, 0, 0])

# Time Complexity: O(n)

OK: pes_1234
OK: pes_zero


## Section 3 — FAANG Hard (Optional / Bonus)

### Task 3.1: Sliding Window Maximum (Deque)

**FAANG Classic:** maintain a decreasing deque of indices.

# HINT:
- Pop from left if out of window
- Pop from right while current value is larger

**Explain:** Why is this amortized O(n)?

**Answer:** Each index is added once and can be removed at most once (from left or right). So total deque operations across the whole loop are linear -> O(n).

In [24]:
def max_sliding_window(nums, k):
    # TODO
    dq = deque()

    out = []

    for i, x in enumerate(nums):
      while dq and dq[0] <= i-k:
        dq.popleft()
      while dq and nums[dq[-1]] <= x:
        dq.pop()

      dq.append(i)

      if i >= k - 1:
        out.append(nums[dq[0]])
    return out

check('msw', max_sliding_window([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7])

OK: msw


### Task 3.2: LRU Cache (Hash Map + Doubly Linked List)

**FAANG Hard/Gotcha:** You must get O(1) `get` and `put`.

# HINT:
- Dict: key -> node
- Doubly linked list: move-to-front on access

You can implement this later if you run out of time today.

---

## Submission Checklist
- All TODOs completed
- All tests passing
- Explain prompts answered
- Complexity noted
